In [13]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import EfficientNetB0, preprocess_input
from tensorflow.keras.metrics import MeanAveragePrecision
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

ImportError: cannot import name 'MeanAveragePrecision' from 'tensorflow.keras.metrics' (C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\_tf_keras\keras\metrics\__init__.py)

In [ ]:
# Параметры для обработки изображений
img_height = 512
img_width = 512
batch_size = 64

In [ ]:
dataset_dir = './dataset'
classes = os.listdir(dataset_dir)

print(classes)

['Plane', 'Willow']


In [ ]:
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_datagen.flow_from_directory(
    './dataset',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

validation_generator = test_datagen.flow_from_directory(
    './validDataset/',
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)


print(train_generator.class_indices)

Found 400 images belonging to 2 classes.
Found 411 images belonging to 2 classes.
{'Plane': 0, 'Willow': 1}


In [ ]:
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))
# EfficientNetB3 - модель, которая позволяет кушать больше изображений
# Метрика %Map10
base_model.trainable = False

inputs = tf.keras.layers.Input(shape=(512, 512, 3))

x = base_model(inputs, training=False)
# Преобразование каритнок из датасета в набор векторов, чтобы хранить их в моделе, в дальнейшем их необходимо сравнивать
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(64, activation="softmax", name="last_dense")(x)
outputs = tf.keras.layers.Dense(2, activation='softmax')(x)
# На выходе необходимо получать не именя классов, а именно вектора изображений, которые и будут сравниваться
model = tf.keras.models.Model(inputs, outputs)

map10 = MeanAveragePrecision(num_classes=2, average='macro', top_k=10)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[map10])

In [ ]:
if os.path.exists('newModel.keras'):
    model = tf.keras.models.load_model('newModel.keras')
else:
    model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=10,
        validation_data=validation_generator,
        validation_steps=len(validation_generator)
    )

model.save('newModel.keras')

In [ ]:
model_load = tf.keras.models.load_model('newModel.keras')

loss, acc = model.evaluate(validation_generator, verbose=2)
print("Validation accuracy:", acc)

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


7/7 - 31s - 4s/step - Precision: 0.9538 - loss: 0.5165
Validation accuracy: 0.9537712931632996


In [ ]:
def load_and_preprocess_image(image_path):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(512, 512))
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_batch = np.expand_dims(img_array, axis=0)
    return preprocess_input(img_batch)

# Загружаем и подготавливаем изображение
new_image = load_and_preprocess_image('boeing.jpg')

In [ ]:
# Выполняем предсказание
preds = model.predict(new_image)
predicted_class_index = np.argmax(preds)

# Получаем имя класса на основе индекса
predicted_class_name = classes[predicted_class_index]

print("Predicted class:", predicted_class_name)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Predicted class: Plane


In [ ]:
def find_similar_images(class_index, n=10):
    similar_images = []
    # Заменить class_index на class_name
    for root, dirs, files in os.walk(f'dataset/{class_index}'):
        for file in files[:n]:
            image_path = os.path.join(root, file)
            similar_images.append(image_path)
            print(image_path)

    print(similar_images)        
    return similar_images
# CSV файл

In [ ]:
similar_images = find_similar_images(predicted_class_name)

dataset/Plane\00144844bdaf8351.jpg
dataset/Plane\006130a7fc2e99cc.jpg
dataset/Plane\014c0014125d4e83.jpg
dataset/Plane\01eabb949fec8850.jpg
dataset/Plane\01ff1d93e2883373.jpg
dataset/Plane\02356f8e29b267d0.jpg
dataset/Plane\0237665a2007ab89.jpg
dataset/Plane\029524bb3c0a5ac4.jpg
dataset/Plane\029691d37fbe20af.jpg
dataset/Plane\02b395ca0ae85a3d.jpg
['dataset/Plane\\00144844bdaf8351.jpg', 'dataset/Plane\\006130a7fc2e99cc.jpg', 'dataset/Plane\\014c0014125d4e83.jpg', 'dataset/Plane\\01eabb949fec8850.jpg', 'dataset/Plane\\01ff1d93e2883373.jpg', 'dataset/Plane\\02356f8e29b267d0.jpg', 'dataset/Plane\\0237665a2007ab89.jpg', 'dataset/Plane\\029524bb3c0a5ac4.jpg', 'dataset/Plane\\029691d37fbe20af.jpg', 'dataset/Plane\\02b395ca0ae85a3d.jpg']
